In [89]:
!aws codeartifact login --tool pip --domain cirrus-ml-ds-domain --domain-owner 813736554012 --repository cirrus-ml-ds-shared-repo
!pip install awswrangler --quiet
!pip install lightgbm --quiet
!pip install category_encoders --quiet
!pip install imbalanced-learn --quiet
# !pip install sagemaker --quiet

Successfully configured pip to use AWS CodeArtifact repository https://cirrus-ml-ds-domain-813736554012.d.codeartifact.eu-north-1.amazonaws.com/pypi/cirrus-ml-ds-shared-repo/ 
Login expires in 12 hours at 2022-08-12 19:17:14+00:00


In [90]:
import boto3
import awswrangler as wr
import pandas as pd
from lightgbm import LGBMClassifier
import numpy as np
from utils_determine_feature_type import determine_feature_data_types
from utils_split import make_splits, split_data, make_subsplit
from utiles_preprocessing import preprocessing_pipeline_step

boto3.setup_default_session(region_name="eu-north-1")

In [91]:
df = wr.athena.read_sql_query('SELECT * FROM "customerone_mock_data_rl"."master";', 
                              database="customerone_mock_data_rl",
                              workgroup="dev-athena-workgroup",
                             )
#                               workgroup="dev-ds-athena-workgroup",
df

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0
0,100102,2021-05-28,<NA>,63,48,M,Macktown,British Indian Ocean Territory (Chagos Archipe...,0,Good,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
1,100103,2021-10-09,<NA>,48,6,M,New Sarah,Aruba,2,Good,...,NaN,NaN,1.483871,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
2,100109,2021-07-29,<NA>,46,10,F,Lake Christophertown,Martinique,4,Bad,...,1.000000,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
3,100117,2021-08-18,<NA>,63,78,M,Meganland,Georgia,4,Bad,...,1.625000,NaN,NaN,1.733333,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
4,100121,2021-10-05,<NA>,32,68,U,Lake Anthonymouth,Madagascar,1,Good,...,5.166667,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40204,100950,2021-08-05,<NA>,67,44,U,Kylemouth,Sao Tome and Principe,4,Bad,...,NaN,NaN,1.000000,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed
40205,100955,2021-07-07,<NA>,53,11,U,East Jamesstad,Guinea-Bissau,1,Good,...,NaN,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40206,10097,2021-08-05,<NA>,60,19,M,West Connieport,Australia,2,Bad,...,1.000000,NaN,1.000000,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40207,100976,2021-09-04,<NA>,46,74,F,Hannahbury,Bulgaria,1,Good,...,1.127273,NaN,1.612903,NaN,0,0,193.435752,0.0,0.0,xsell_cust_voice_to_fixed


In [92]:
target_col_name = 'tgt_xsell_cust_voice_to_fixed'


# Train

In [94]:
# skipped sample_training_df()
# skipped drop_invalid_features()
# skipped filter_features()

# skipped to_pandas() and just filtered the null target rows:
df_training = df[df['tgt_xsell_cust_voice_to_fixed'].notnull()]
df_training_idx = df_training.reset_index(drop=True)
print(f"{df_training.shape}, {df_training_idx.shape}")

(5896, 74), (5896, 74)


In [95]:
df_training_idx

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0
0,100314,2021-08-12,0,62,98,U,East Stephen,Bhutan,2,Bad,...,1.0,NaN,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
1,100314,2021-08-12,0,62,98,U,East Stephen,Bhutan,2,Bad,...,1.0,NaN,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
2,100318,2021-05-25,0,47,33,U,Nathanielfurt,Korea,3,Bad,...,NaN,NaN,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed
3,100318,2021-05-25,1,47,33,U,Nathanielfurt,Korea,3,Bad,...,NaN,NaN,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed
4,100620,2021-09-02,0,26,70,F,West Samantha,Qatar,0,Bad,...,NaN,NaN,1.00000,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,100249,2021-06-10,0,24,91,U,South Marissa,Qatar,3,Bad,...,NaN,NaN,1.00000,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed
5892,100249,2021-06-10,1,24,91,U,South Marissa,Qatar,3,Bad,...,NaN,NaN,1.00000,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed
5893,100271,2021-07-27,0,25,57,M,North Pam,Mongolia,2,Bad,...,1.0,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
5894,100487,2021-06-07,0,39,37,M,North Dustin,Afghanistan,4,Good,...,NaN,NaN,1.83871,NaN,0,0,278.592307,0.0,0.0,xsell_cust_voice_to_fixed


# features

In [96]:
spine_params_determine_feature_data_types = {'keys': ['customer_id'],
                                             'date_column': 'current_dt',
                                             'product_holdings_filter': {'product_category': 'fixedbroadband'},
                                             'is_deepsell': 'N'}

target_params_determine_feature_data_types = {'target_variable_column': 'tgt_xsell_cust_voice_to_fixed'
    , 'lead_time_window': '1d'
    , 'target_window': '45d'
    , 'product_activation_filter': {'product_sub_category': 'voice'}
    , 'campaign_keys': ['customer_id']
    , 'campaign_filter': {'campaign_name': ["C-452-O-06 Korsförsäljning Telia Life 2.0 - Sälja mobilt"
        , "b2c_cross-sell_pp_crossSellPpToBb"
        , "b2c_cross-sell_pp_crossSellPpToBb_oldContent"
        , "b2c_cross-sell_pp_crossSellPpToBb_REM1"
        , "C-700-O-03 Cross-sell Mobile to Broadband customers (TM only)"
        , "C-700-O-01 Cross-sell Mobile to Broadband customers (A)"
        , "C-752-O GEOF 2021 - X-sell PP"
        , "C-652-O Black Friday Erbjudande 2019  Mobilt till BB-kund - activity 1"
        , "C-700-O-02 Cross-sell Mobile to Broadband customers (B)"
        , "b2c_cross-sell_Pp_PpToBb_default"
        , "C-752-O GEOF 2021 - Xsell PP"
        , "b2c_cross-sell_Pp_PpToBb_simOnly"
        , "b2c_cross-sell_Pp_PpToBb_samS215G"
        , "b2c_cross-sell_Pp_PpToBb_iphone12Mini"
        , "b2c_cross-sell_Pp_PpToBb_iphoneSE"
        , "b2c_cross-sell_Pp_PpToBb_iphone12"
        , "b2c_cross-sell_Pp_PpToBb_samS20FE5G"
        , "b2c_cross-sell_Pp_PpToBb_sonyXp10lll"
        , "b2c_cross-sell_Pp_PpToBb_default_short_8pm"
        , "b2c_cross-sell_Pp_PpToBb_default_8pm"]
        , 'customer_actioned_flg_column': {'Email': 'actioned_ind'}}}

feature_dict = determine_feature_data_types(df_training,
                                            spine_params_determine_feature_data_types,
                                            target_params_determine_feature_data_types)
print(f"len(feature_dict)={len(feature_dict)}, \nlen(feature_dict['numeric'])={len(feature_dict['numeric'])}, \nlen(feature_dict['categorical'])={len(feature_dict['categorical'])}")

features = feature_dict['categorical'] + feature_dict['numeric']

len(feature_dict)=3, 
len(feature_dict['numeric'])=62, 
len(feature_dict['categorical'])=3


In [97]:
# Splitting data
split_train_test = make_splits(df_training_idx, target_col_name)

# print(f"\n\n{df_training.shape}, {df_training_idx.shape}, {split_train_test.shape}\n")
print(f"split_train_test['split'].value_counts(dropna=False):\n {split_train_test['split'].value_counts(dropna=False)}\n")
print(f"df_training_idx['split'].value_counts(dropna=False):\n {df_training_idx['split'].value_counts(dropna=False)}\n")
print(f"split_train_test['iteration_id'].value_counts(dropna=False):\n {split_train_test['iteration_id'].value_counts(dropna=False)}")

split_train_test



_extract_group_column_arg: group_col_name=customer_id
_extract_group_column_arg: target_col_name=tgt_xsell_cust_voice_to_fixed

Split 0
	train: 	2021-05-25 -> 2021-08-14	shape: (4654, 76)	ratio: 0.79
	test: 	2021-08-15 -> 2021-09-09	shape: (1242, 76)	ratio: 0.21
split_train_test['split'].value_counts(dropna=False):
 TRAIN    4654
TEST     1242
Name: split, dtype: int64

df_training_idx['split'].value_counts(dropna=False):
 NaN    5896
Name: split, dtype: int64

split_train_test['iteration_id'].value_counts(dropna=False):
 0    5896
Name: iteration_id, dtype: int64


,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0,iteration_id,split
0,100314,2021-08-12,0,62,98,U,East Stephen,Bhutan,2,Bad,...,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
1,100314,2021-08-12,0,62,98,U,East Stephen,Bhutan,2,Bad,...,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
2,100318,2021-05-25,0,47,33,U,Nathanielfurt,Korea,3,Bad,...,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
3,100318,2021-05-25,1,47,33,U,Nathanielfurt,Korea,3,Bad,...,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
4,100620,2021-09-02,0,26,70,F,West Samantha,Qatar,0,Bad,...,1.00000,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,100249,2021-06-10,0,24,91,U,South Marissa,Qatar,3,Bad,...,1.00000,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
5892,100249,2021-06-10,1,24,91,U,South Marissa,Qatar,3,Bad,...,1.00000,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
5893,100271,2021-07-27,0,25,57,M,North Pam,Mongolia,2,Bad,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
5894,100487,2021-06-07,0,39,37,M,North Dustin,Afghanistan,4,Good,...,1.83871,NaN,0,0,278.592307,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN


In [98]:
splitter_args = {'test_size': 0.1, 'random_state': 42}
subsplit_params = {"iteration_col_name": 'iteration_id', "split_col_name": "split", 'source_split': 'TRAIN',
                   'target_splits': ['TRAIN', 'CAL']}

split_train_test_cal = make_subsplit(split_train_test, subsplit_params, splitter_args, target_col_name)
print(f"\n\nsplit_train_test_cal['split'].value_counts(dropna=False):\n {split_train_test_cal['split'].value_counts(dropna=False)}")
split_train_test_cal



_extract_group_column_arg: group_col_name=None
_extract_group_column_arg: target_col_name=tgt_xsell_cust_voice_to_fixed


split_train_test_cal['split'].value_counts(dropna=False):
 TRAIN    4188
TEST     1242
CAL       466
Name: split, dtype: int64


,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0,iteration_id,split
0,100314,2021-08-12,0,62,98,U,East Stephen,Bhutan,2,Bad,...,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
1,100314,2021-08-12,0,62,98,U,East Stephen,Bhutan,2,Bad,...,NaN,1.0,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,CAL
2,100318,2021-05-25,0,47,33,U,Nathanielfurt,Korea,3,Bad,...,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
3,100318,2021-05-25,1,47,33,U,Nathanielfurt,Korea,3,Bad,...,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
4,100620,2021-09-02,0,26,70,F,West Samantha,Qatar,0,Bad,...,1.00000,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,100249,2021-06-10,0,24,91,U,South Marissa,Qatar,3,Bad,...,1.00000,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
5892,100249,2021-06-10,1,24,91,U,South Marissa,Qatar,3,Bad,...,1.00000,NaN,0,0,116.595360,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
5893,100271,2021-07-27,0,25,57,M,North Pam,Mongolia,2,Bad,...,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
5894,100487,2021-06-07,0,39,37,M,North Dustin,Afghanistan,4,Good,...,1.83871,NaN,0,0,278.592307,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN


In [117]:
%load_ext autoreload 
%autoreload 2
from test_train_splitter import OutOfTimeSplitter

import os
import numpy as np


def to_list_func(item):
    return [item] if not isinstance(item, list) else item


def kfold_split_data(data, estimator, tuning_params, feature_col_names, splitting_params, target_col_name):
    iteration_col_name = splitting_params.get("iteration_col_name", "iteration_id")
    split_col_name = splitting_params.get("split_col_name", "split")
    
    cv = OutOfTimeSplitter(tuning_params.get('cv_args').get('test_size'), 
                           tuning_params.get('cv_args').get('date_col'), 
                           tuning_params.get('cv_args').get('n_splits'),
                           tuning_params.get('cv_args').get('random_state'), 
                           tuning_params.get('cv_args').get('verbose'))
    print(f"\n\n cv={cv}")
    print(f"test_size={tuning_params.get('cv_args').get('test_size')}, date_col={tuning_params.get('cv_args').get('date_col')}, n_splits={tuning_params.get('cv_args').get('n_splits')}, random_state={tuning_params.get('cv_args').get('random_state')}, verbose={tuning_params.get('cv_args').get('verbose')}")
    
    splits_to_fit = to_list_func(tuning_params.get("splits_to_fit", "TRAIN"))
    group_col_name = tuning_params.get("cv_split_group_col_name", None)

    print("\n\n************ start of _find_best_params_per_fold")
    train_index = data[split_col_name].isin(splits_to_fit)

    # keep all columns in X_train in case any of them are needed to make the split
    partition_train = data.loc[train_index].copy()
    X_train = partition_train
    y_train = partition_train[target_col_name]
    

    if hasattr(cv, "split") and not isinstance(cv, str):
        group_col = (
            partition_train[group_col_name]
            if group_col_name is not None
            else group_col_name
        )
        print(f"group_col={group_col}")
        tuning_params["cv"] = list(cv.split(X=X_train, y=y_train, groups=group_col))
    print(f"\n\ntuning_params[cv] len = {len(tuning_params['cv'])}, type(tuning_params['cv']) = {type(tuning_params['cv'])}") 
    print(f"len[0]={len(tuning_params['cv'][0])}, len[0][0]={len(tuning_params['cv'][0][0])} , len[0][1]={len(tuning_params['cv'][0][1])} ")
    print(f"len[1]={len(tuning_params['cv'][1])}, len[1][0]={len(tuning_params['cv'][1][0])} , len[1][1]={len(tuning_params['cv'][1][1])} ")
    print(f"len[2]={len(tuning_params['cv'][2])}, len[2][0]={len(tuning_params['cv'][2][0])} , len[2][1]={len(tuning_params['cv'][2][1])} ")
    

    # use only selected features for tuning 
    X_train = X_train[feature_col_names]
    print(f"\nX_train.shape = {X_train.shape}")
#     X_train.to_csv(f'train_x.csv', )

    y_train = y_train.astype(int)
    print(f"\ntest[0]={y_train[0]}, type(y_train[0])={type(y_train[0])}")
    print("\n\n")

    for k in range(0, len(tuning_params["cv"])):
        print(f"k={k}")
        X_train_fold, X_test_fold = X_train.iloc[tuning_params['cv'][k][0]], X_train.iloc[tuning_params['cv'][k][1]]
        y_train_fold, y_test_fold = y_train.iloc[tuning_params['cv'][k][0]], y_train.iloc[tuning_params['cv'][k][1]]
        print(f"X_train_fold_{k}={X_train_fold.shape}, X_test_fold_{k}={X_test_fold.shape}")
        print(f"y_train_fold_{k}={y_train_fold.shape}, X_test_fold_{k}={y_train_fold.shape}")
# #        os.makedirs(f'{base_dir}/train/{fold_idx}', exist_ok=True)
# #        np.savetxt(f'{base_dir}/train/{fold_idx}/train_x.csv', X_train, delimiter=',')
# #        np.savetxt(f'{base_dir}/train/{fold_idx}/train_y.csv', y_train, delimiter=',')

# #        os.makedirs(f'{base_dir}/test/{fold_idx}', exist_ok=True)
# #        np.savetxt(f'{base_dir}/test/{fold_idx}/test_x.csv', X_test, delimiter=',')
# #        np.savetxt(f'{base_dir}/test/{fold_idx}/test_y.csv', y_test, delimiter=',')
        os.makedirs(f'kfold/train/{k}', exist_ok=True)
        X_train_fold.to_csv(f'kfold/train/{k}/train_x.csv')
        y_train_fold.to_csv(f'kfold/train/{k}/train_y.csv')

        os.makedirs(f'kfold/test/{k}', exist_ok=True)
        X_test_fold.to_csv(f'kfold/test/{k}/test_x.csv')
        y_test_fold.to_csv(f'kfold/test/{k}/test_y.csv')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [118]:
tuning_params = {'tuner': 'sklearn.model_selection.RandomizedSearchCV', 
                 'tuner_args': {'scoring': 'roc_auc', 'verbose': 2, 'n_iter': 10, 
                                'param_distributions': {'preprocessing__categorical__encoder__min_samples_leaf': [100, 500, 1000, 5000, 10000],
                                                         'preprocessing__categorical__encoder__smoothing': [1.0, 5.0, 10.0, 25.0], 
                                                         'over_sampler__sampling_strategy': [0.5, 0.35, 0.25, 0.15, 0.1, 'not minority'], 
                                                         'estimator__boosting_type': ['gbdt', 'dart'], 
                                                         'estimator__num_leaves': [20, 31, 50, 100], 
                                                         'estimator__min_child_samples': [100, 250, 500, 1000], 
                                                         'estimator__min_child_weight': [0.01, 0.001], 
                                                         'estimator__max_depth': [5, 10, 15, -1], 
                                                         'estimator__learning_rate': [0.2, 0.15, 0.1, 0.05, 0.01], 
                                                         'estimator__n_estimators': [50, 100, 150], 
                                                         'estimator__colsample_bytree': [0.8, 1.0], 
                                                         'estimator__subsample': [0.8, 1.0], 
                                                         'estimator__subsample_freq': [1], 
                                                         'estimator__reg_alpha': [0.0, 0.1],
                                                         'estimator__reg_lambda': [0.0, 0.1],
                                                         'estimator__min_split_gain': [0.0, 0.1],
                                                         'estimator__class_weight': ["balanced", []],
                                                        }}, 
                 'cv': 'test_train_splitter.OutOfTimeSplitter', 
                 'cv_args': {'n_splits': 3, 'test_size': 0.45, 'date_col': 'current_dt', 'random_state': 42, 'verbose': 1}}

features = feature_dict['categorical'] + feature_dict['numeric']

splitting_params = {'splitter': 'test_train_splitter.OutOfTimeSplitter', 
                    'splitter_args': {'n_splits': 1, 'test_size': 0.2, 'date_col': 'current_dt', 'verbose': 1, 'random_state': 42}, 
                    'group_col_name': 'customer_id', 
                    'iteration_col_name': 'iteration_id', 
                    'split_col_name': 'split'}

target = 'tgt_xsell_cust_voice_to_fixed'

kfold_split_data(split_train_test_cal, estimator_pipeline, tuning_params, features, splitting_params, target_col_name)




 cv=<test_train_splitter.OutOfTimeSplitter object at 0x7f7dbc1fcbd0>
test_size=0.45, date_col=current_dt, n_splits=3, random_state=42, verbose=1


************ start of _find_best_params_per_fold


 3-OutOfTimeSplitter.split > n_splits = 3

 OutOfTimeSplitter.split > split=0
	train: 	2021-05-25 -> 2021-07-08	shape: (2275, 76)	ratio: 0.54
	test: 	2021-07-09 -> 2021-07-19	shape: (666, 76)	ratio: 0.16

 OutOfTimeSplitter.split > train_idx=Int64Index([   1,    2,    5,    6,    9,   13,   15,   16,   20,   21,
            ...
            4175, 4176, 4179, 4180, 4181, 4182, 4183, 4184, 4186, 4187],
           dtype='int64', length=2275), test_idx=Int64Index([   3,    4,   14,   19,   35,   36,   54,   55,   61,   62,
            ...
            4153, 4155, 4156, 4157, 4158, 4161, 4163, 4164, 4171, 4172],
           dtype='int64', length=666)

 OutOfTimeSplitter.split > split=1
	train: 	2021-05-25 -> 2021-07-18	shape: (2883, 76)	ratio: 0.69
	test: 	2021-07-19 -> 2021-08-01	shape: (686, 76)

# Rest of preprocessing

In [100]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder

class PandasSimpleImputer(SimpleImputer):
    """
    A wrapper around `SimpleImputer` to return data frames with columns.
    """
    def fit(self, X, y=None):
        print("\n\n ****** PandasSimpleImputer > fit ******** ")
        print(f" ****** PandasSimpleImputer > fit ******** X.shape={X.shape}, y.shape={y.shape}")    
        self.columns = X.columns
        return super().fit(X, y)

    def transform(self, X):
        print(" ****** PandasSimpleImputer > transform ******** ")
        print(f" ****** PandasSimpleImputer > transform ******** X.shape={X.shape}")
        return pd.DataFrame(super().transform(X), columns=self.columns, index=X.index)



preprocessing = []

# preproc_name = 'categorical'
# steps = {'impute': PandasSimpleImputer(strategy='most_frequent'), 'encoder': }
pipeline_cat = Pipeline(steps=[('impute', PandasSimpleImputer(strategy='most_frequent')),
                               ('encoder', TargetEncoder())])
preprocessing.append(('categorical', pipeline_cat, feature_dict['categorical']))

# preproc_name = 'numerical'
# steps = {'imputer': PandasSimpleImputer()}
pipeline_num = Pipeline(steps=[('imputer', PandasSimpleImputer())])
# preprocessing.append(('numerical', pipeline_num, feature_dict['numeric'])) ## is it correct?
preprocessing.append(('numerical', pipeline_num, feature_dict['categorical'])) ## is it correct?
preprocessing_step = ColumnTransformer(preprocessing)
preprocessing_step


ColumnTransformer(transformers=[('categorical',
                                 Pipeline(steps=[('impute',
                                                  PandasSimpleImputer(strategy='most_frequent')),
                                                 ('encoder', TargetEncoder())]),
                                 ['rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg',
                                  'rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
                                  'dmgrphc_b_zip_code_cd']),
                                ('numerical',
                                 Pipeline(steps=[('imputer',
                                                  PandasSimpleImputer())]),
                                 ['rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg',
                                  'rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
                                  'dmgrphc_b_zip_code_cd'])])

In [101]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
# from sklearn.pipeline import Pipeline


estimator_pipeline = Pipeline(steps=[('over_sampler', SMOTE(random_state=42)),
                                     ('estimator', LGBMClassifier())])

# add_step = {'step':preprocessing_step, 'index': 0, 'step_name':'preprocessing'}

# estimator_pipeline.steps.insert(add_step.get('index'), (add_step.get('step_name'), add_step.get('step')))
estimator_pipeline.steps.insert(0, ('preprocessing', preprocessing_step))

estimator_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   PandasSimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder())]),
                                                  ['rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg',
                                                   'rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
                                                   'dmgrphc_b_zip_code_cd']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   PandasSimpleImputer())]),
                

# Cross validation

In [ ]:

from sagemaker.workflow.parameters import ParameterInteger, ParameterString

training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.xlarge")

# training step for generating model artifacts (Specify the training container image URI)
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region="eu-north-1",
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)
model_name = "xsell_cust_voice_to_fixed"

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
#     output_path=model_path,
#     base_job_name=f"{base_job_prefix}/abalone-train",
#     sagemaker_session=sagemaker_session,
    role=env_data["TrainingRole"],
    subnets=network_config.subnets,
    security_group_ids=network_config.security_group_ids,
    encrypt_inter_container_traffic=True,
    enable_network_isolation=False,
    volume_kms_key=env_data["EbsKmsKeyArn"],
    output_kms_key=env_data["S3KmsKeyId"]
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)
step_train = TrainingStep(
    name="TrainAbaloneModel",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri, content_type="text/csv",),
        # "validation": TrainingInput(s3_data=step_process.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,content_type="text/csv",),
        # "test": TrainingInput(s3_data=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri, content_type="text/csv", ),
    },
)


In [ ]:
best_tuning_score

In [ ]:
tuned_parameters